# Mammary Adenoma vs Adenocarcinoma 추출 PoC (v1)
이 노트북은 앞서 설명한 단계별 필터링 절차를 적용하여 `Data/조직검사 결과 매칭(2024)_utf8_pruned.csv`에서 **Mammary Adenoma**와 **Mammary Adenocarcinoma**를 구분합니다.
<sub>규칙 기반 1차 필터와 수동 검토 루프를 묶어 약한 감독(weak supervision) 방식으로 레이블링 비용을 낮추는 PoC 흐름을 검증합니다.</sub>

In [ ]:
import pandas as pd
from pathlib import Path
import pyarrow  # ensure parquet engine is available

pd.options.display.max_colwidth = 120

DATA_PATH = Path("../../Data/조직검사 결과 매칭(2024)_utf8_pruned.parquet")
assert DATA_PATH.exists(), f"Missing data file: {DATA_PATH}"

raw_df = pd.read_parquet(DATA_PATH, engine="pyarrow")
raw_df.head()


## 1. 스키마와 초기 건수 확인

<sub>스키마와 총 건수를 미리 확인해야 이후 필터링에서 열 이름 오타나 누락으로 인한 정보 손실을 방지할 수 있습니다.</sub>

In [ ]:
raw_df.info()

## 2. 텍스트 정규화 도우미
소문자 변환, HTML `<br>` 제거, 공백 정리, 그리고 느슨한 매칭을 위한 알파벳 전용 보조 문자열을 생성합니다.
<sub>서술문이 길거나 HTML 줄바꿈이 섞인 경우도 일관된 형태로 맞추기 위해 정규화를 먼저 수행합니다 (예: `Mammary adenocarcinoma<br>complete excision` → `mammary adenocarcinoma complete excision`).</sub>

In [ ]:
import re

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = s.replace('<br>', ' ')
    s = re.sub(r"[^a-z0-9\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def alpha_only(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

norm_cols = ['DIAGNOSIS', 'GROSS_FINDINGS', 'COMMENTS', 'MICROSCOPIC_FINDINGS']
norm_df = raw_df.copy()
for col in norm_cols:
    norm_df[f"{col}_norm"] = norm_df[col].apply(normalize_text)
    norm_df[f"{col}_alpha"] = norm_df[col].apply(alpha_only)

norm_df[[c for c in norm_df.columns if c.endswith('_norm')]].head()

## 3. 키워드 규칙
Adenoma와 Adenocarcinoma 각각에 대해 확정 매칭용 기본 패턴과 다른 컬럼의 보조 컨텍스트 용어를 정의합니다.
<sub>자유 서술식으로 입력된 진단명을 포착하기 위해 강한 패턴과 약한 보조 컨텍스트를 분리해 관리하고, 두 진단군을 분리하는 데 핵심이 되는 malignant 표현(`carcinoma`, `malignant`, `invasive` 등)을 별도로 모읍니다.</sub>

In [ ]:
primary_patterns = {
    'adenoma': [
        r"mammary adenom",
        r"mamary adenom",  # common misspelling
        r"mammary benign adenom",
        r"apocrine or mammary adenom",
        r"mammary adenom[ae]",
        r"mammary\s+gland\s+adenom",
    ],
    'adenocarcinoma': [
        r"mammary adenocarcin",
        r"mammary carcinoma",
        r"mammary\s+gland\s+adenocarcin",
        r"mammary\s+gland\s+carcinoma",
        r"mammary malignant tumor",
    ],
}

# 전문가가 확실한 표현을 추가할 때 사용 (예: r"mammary gland adenoma")
custom_primary_patterns = {
    'adenoma': [],
    'adenocarcinoma': [],
}

# 지원 컬럼별 기본 보조 키워드
support_keywords = {
    'adenoma': {
        'COMMENTS': ["adenoma", "유선", "apocrine", "benign", "adenomatous"],
        'MICROSCOPIC_FINDINGS': ["adenoma", "lobule", "capsulated", "ductal", "benign"],
        'GROSS_FINDINGS': ["mammary", "乳腺", "mamary", "mass", "nodule"],
    },
    'adenocarcinoma': {
        'COMMENTS': ["adenocarcinoma", "carcinoma", "malignant", "invasive", "metastatic", "anaplastic"],
        'MICROSCOPIC_FINDINGS': ["carcinoma", "adenocarcinoma", "malignant", "invasive", "anaplastic", "pleomorphic"],
        'GROSS_FINDINGS': ["mammary", "mamary", "ulcer", "hemorrhage", "necrosis", "irregular"],
    },
}

# 전문가가 보조 키워드를 추가할 때 사용 (컬럼별 리스트를 확장)
custom_support_keywords = {
    'adenoma': {
        'COMMENTS': [],
        'MICROSCOPIC_FINDINGS': [],
        'GROSS_FINDINGS': [],
    },
    'adenocarcinoma': {
        'COMMENTS': [],
        'MICROSCOPIC_FINDINGS': [],
        'GROSS_FINDINGS': [],
    },
}

# 느슨한 공존 판정을 위한 기본 용어
loose_terms = {
    'adenoma': ["mammary", "mamary"],
    'adenocarcinoma': ["mammary", "mamary"],
}

all_primary_patterns = {
    label: pats + custom_primary_patterns.get(label, [])
    for label, pats in primary_patterns.items()
}

merged_support_keywords = {
    label: {
        col: keywords + custom_support_keywords.get(label, {}).get(col, [])
        for col, keywords in support_map.items()
    }
    for label, support_map in support_keywords.items()
}


## 4. 점수 산정 함수
- 진단 컬럼에 확정 패턴 매칭 시 +2 (adenoma/adenocarcinoma 별도 집계)
- 알파벳 전용 문자열에서 'mammary/mamary'와 핵심 진단 토큰(adenoma vs carcinoma)이 함께 등장하면 +1
- 지원 컬럼에 보조 키워드가 있으면 컬럼당 +1
- 최종 라벨: 점수가 더 큰 쪽을 우선하며, 둘 다 0이면 `none`, 동률이면 `uncertain`
<sub>양성과 악성을 동일한 `mammary` 해부 부위 안에서 구분하기 위해 두 집합을 병렬로 점수화합니다. 동시에 등장하거나 애매한 경우를 `uncertain`으로 남겨 수동 검토 대상을 명확히 합니다.</sub>

In [ ]:

import pandas as pd

mammary_context_terms = ['mammary', 'mamary']
mammary_context_korean = ['유선', '乳腺']

def has_mammary_context(row):
    raw_fields = ' '.join(
        str(row.get(col, ''))
        for col in ['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS']
        if isinstance(row.get(col, ''), str)
    ).lower().replace('<br>', ' ')
    if any(term in row['DIAGNOSIS_norm'] for term in mammary_context_terms):
        return True
    return any(term in raw_fields for term in mammary_context_terms + mammary_context_korean)

def score_row(row):
    scores = {'adenoma': 0, 'adenocarcinoma': 0}
    diag_norm = row['DIAGNOSIS_norm']
    diag_alpha = row['DIAGNOSIS_alpha']
    diag_tokens = set(diag_alpha.split())

    for label in scores:
        # primary patterns
        if any(re.search(pat, diag_norm) for pat in all_primary_patterns[label]):
            scores[label] += 2

        # loose co-occurrence within diagnosis
        if label == 'adenoma':
            if diag_tokens.intersection(loose_terms[label]) and 'adenoma' in diag_tokens:
                scores[label] += 1
        else:
            if diag_tokens.intersection(loose_terms[label]) and (
                'carcinoma' in diag_tokens or 'adenocarcinoma' in diag_tokens
            ):
                scores[label] += 1

        # supporting columns
        for col, keywords in merged_support_keywords[label].items():
            text = row[f"{col}_norm"]
            if any(kw in text for kw in keywords):
                scores[label] += 1

    mammary_ok = has_mammary_context(row)

    if scores['adenoma'] == 0 and scores['adenocarcinoma'] == 0:
        label = 'none'
    elif not mammary_ok:
        label = 'uncertain'
    elif scores['adenoma'] > scores['adenocarcinoma']:
        label = 'mammary_adenoma'
    elif scores['adenocarcinoma'] > scores['adenoma']:
        label = 'mammary_adenocarcinoma'
    else:
        label = 'uncertain'

    return pd.Series({
        'score_adenoma': scores['adenoma'],
        'score_adenocarcinoma': scores['adenocarcinoma'],
        'label': label,
    })

scores_df = norm_df.apply(score_row, axis=1)
norm_df = pd.concat([norm_df, scores_df], axis=1)

norm_df[['DIAGNOSIS', 'score_adenoma', 'score_adenocarcinoma', 'label']].head()


In [ ]:
# adenoma로 분류된 행 미리보기 (전수 확인용)
adenoma_rows = norm_df[norm_df['label'] == 'mammary_adenoma']
adenoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenoma']]


In [ ]:
# adenocarcinoma로 분류된 행 미리보기 (전수 확인용)
adenocarcinoma_rows = norm_df[norm_df['label'] == 'mammary_adenocarcinoma']
adenocarcinoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenocarcinoma']]


## 5. 경계 사례 검토
점수가 동률이거나 둘 다 낮은 행을 확인하여 잘못된 분류를 보정합니다.
<sub>예: `diagnosis`에 adenoma, `micro`에 carcinoma가 언급된 혼합 표현을 사람이 빠르게 검토해 룰셋을 개선합니다.</sub>

In [ ]:
uncertain = norm_df[norm_df['label'].isin(['uncertain', 'none'])]
uncertain[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenoma', 'score_adenocarcinoma']].head(10)


## 6. 종(개/고양이) 판정
<sub>숫자 표현(예: `두 개`, `3 개의`)에 등장하는 한글 `개`를 개체로 오인하지 않도록, 종을 암시하는 문맥/품종/영문 표현이 있을 때만 점수를 부여합니다. 확신이 어려우면 `uncertain`으로 남깁니다.</sub>


In [ ]:
species_patterns = {
    'dog': [
        r"canin",
        r"dog(s)?",
        r"개에서",
        r"개\s*환자",
        r"견",
        r"강아지",
    ],
    'cat': [
        r"felin",
        r"cat(s)?",
        r"고양이에서",
        r"고양이\s*환자",
        r"냥이",
    ],
}

breed_keywords = {
    'dog': [
        'poodle', 'toy', 'miniature', 'retriever', 'bulldog', 'beagle', 'shihtzu', 'shih', 'tzu',
        'yorkshire', 'maltese', 'pom', 'pomeranian', 'spitz', 'jindo', 'shepherd', 'collie',
    ],
    'cat': [
        'persian', 'ragdoll', 'bengal', 'munchkin', 'siam', 'siamese', 'fold', 'norwegian',
        'russian', 'blue', 'sphynx', 'cat',
    ],
}

def normalize_species_text(row):
    text = ' '.join(
        str(row.get(col, ''))
        for col in ['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS']
        if isinstance(row.get(col, ''), str)
    ).lower().replace('<br>', ' ')
    text = re.sub(r"[^0-9a-z가-힣\s]+", " ", text)
    return re.sub(r"\s+", " ", text).strip()

def score_species(row):
    text = normalize_species_text(row)
    tokens = set(text.split())
    scores = {'dog': 0, 'cat': 0}

    for label, patterns in species_patterns.items():
        for pat in patterns:
            if re.search(pat, text):
                scores[label] += 2

    for label, keywords in breed_keywords.items():
        for kw in keywords:
            if kw in tokens:
                scores[label] += 1

    if scores['dog'] == 0 and scores['cat'] == 0:
        species_label = 'uncertain'
    elif scores['dog'] > scores['cat']:
        species_label = 'dog'
    elif scores['cat'] > scores['dog']:
        species_label = 'cat'
    else:
        species_label = 'uncertain'

    return pd.Series({
        'score_dog': scores['dog'],
        'score_cat': scores['cat'],
        'species_label': species_label,
    })

species_df = norm_df.apply(score_species, axis=1)
norm_df = pd.concat([norm_df, species_df], axis=1)

norm_df[['DIAGNOSIS', 'species_label', 'score_dog', 'score_cat']].head()


## 7. 양성/악성 판정
<sub>악성과 양성 모두에서 흔히 쓰이는 조직명 표현을 규칙으로 점수화하고, 근거가 약하면 `uncertain`으로 남깁니다.</sub>

In [ ]:
benign_patterns = [
    r"adenoma",
    r"papilloma",
    r"polyp",
    r"fibroma",
    r"lipoma",
    r"osteoma",
    r"chondroma",
    r"hemangioma",
    r"lymphangioma",
    r"leiomyoma",
    r"rhabdomyoma",
    r"neuroma",
    r"fibroadenoma",
    r"mixed tumor",
    r"melanocytoma",
    r"histiocytoma",
]

malignant_patterns = [
    r"carcinoma",
    r"adenocarcinoma",
    r"sarcoma",
    r"fibrosarcoma",
    r"osteosarcoma",
    r"hemangiosarcoma",
    r"liposarcoma",
    r"leiomyosarcoma",
    r"rhabdomyosarcoma",
    r"carcinosarcoma",
    r"malignant mixed tumor",
    r"melanoma",
    r"lymphoma",
    r"mast cell tumor",
    r"histiocytic sarcoma",
]

behavior_support = {
    'benign': ['benign', '양성'],
    'malignant': ['malignant', 'invasive', 'metastatic', 'metastasis', 'anaplastic', '악성'],
}

def concat_norm_text(row):
    return ' '.join(str(row.get(f"{col}_norm", '')) for col in norm_cols)

def score_behavior(row):
    text = concat_norm_text(row)
    scores = {'benign': 0, 'malignant': 0}

    for pat in benign_patterns:
        if re.search(pat, text):
            scores['benign'] += 2
    for pat in malignant_patterns:
        if re.search(pat, text):
            scores['malignant'] += 2

    for label, keywords in behavior_support.items():
        for kw in keywords:
            if kw in text:
                scores[label] += 1

    if scores['benign'] == 0 and scores['malignant'] == 0:
        behavior_label = 'none'
    elif scores['benign'] > scores['malignant']:
        behavior_label = 'benign'
    elif scores['malignant'] > scores['benign']:
        behavior_label = 'malignant'
    else:
        behavior_label = 'uncertain'

    return pd.Series({
        'score_benign': scores['benign'],
        'score_malignant': scores['malignant'],
        'behavior_label': behavior_label,
    })

behavior_df = norm_df.apply(score_behavior, axis=1)
norm_df = pd.concat([norm_df, behavior_df], axis=1)

norm_df[['DIAGNOSIS', 'behavior_label', 'score_benign', 'score_malignant']].head()


## 8. 확실 사례 스냅샷
<sub>두 진단군 각각에서 패턴이 제대로 작동하는지, 해부 부위나 진단 요약이 기대대로 나오는지 빠르게 눈으로 검증합니다.</sub>

In [ ]:
adenoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenoma', 'behavior_label', 'species_label']].head(10)

In [ ]:
adenocarcinoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenocarcinoma', 'behavior_label', 'species_label']].head(10)

## 9. 요약 통계
<sub>라벨 분포와 점수 분포를 보면 규칙 조정이 필요한지 판단할 수 있습니다.</sub>

In [ ]:
total = len(norm_df)
label_counts = norm_df['label'].value_counts()
behavior_counts = norm_df['behavior_label'].value_counts()
species_counts = norm_df['species_label'].value_counts()

print(f"Total rows: {total}")
print("
Mammary label counts:
", label_counts)
print("
Behavior label counts:
", behavior_counts)
print("
Species label counts:
", species_counts)

score_summary = norm_df[['score_adenoma', 'score_adenocarcinoma', 'score_benign', 'score_malignant', 'score_dog', 'score_cat']].describe()
score_summary


## 10. 정제된 데이터셋 저장
<sub>Adenoma/Adenocarcinoma 라벨과 점수를 포함한 CSV를 남겨 후속 분석·모델 학습에서 동일한 결과를 재사용할 수 있습니다.</sub>

In [ ]:
output_cols = [
    'INSP_RQST_NO', 'FOLDER', 'FILE_NAME', 'SITE',
    'DIAGNOSIS', 'GROSS_FINDINGS', 'COMMENTS', 'MICROSCOPIC_FINDINGS',
    'score_adenoma', 'score_adenocarcinoma', 'score_benign', 'score_malignant',
    'score_dog', 'score_cat', 'label', 'behavior_label', 'species_label'
]
curated = norm_df[output_cols].copy()
base_path = Path('./mammary_adenoma_vs_adenocarcinoma_curated')
parquet_path = base_path.with_suffix('.parquet')
csv_path = base_path.with_suffix('.csv')
curated.to_parquet(parquet_path, index=False)
curated.to_csv(csv_path, index=False)
parquet_path, csv_path


## 11. 다음 단계
- 경계 사례 수동 검토 후 keyword 리스트를 보완합니다.
- 희귀 동의어에 대한 recall을 높이려면 TF-IDF나 임베딩 유사도 방식을 고려합니다.
<sub>규칙→수동 리뷰→키워드 보완→모델 전환의 반복 학습 루프를 염두에 둔 로드맵입니다.</sub>